In [2]:
# Importing some basic libraries

import numpy as np
import pandas as pd

# Importing the dataset
data = pd.read_csv('/content/drive/MyDrive/Airline Sentiment Analysis/airline_df_cleaned.csv')
air = pd.read_csv('/content/drive/MyDrive/Airline Sentiment Analysis/airline_df_nlped.csv')
air.head()

,Country,Airline,Review,Review_LS,Sentiment
0,China,Air China,los angeles beijing return. food low quality s...,los angeles beijing return . food quality staf...,0
1,China,Air China,round trip from hong kong to munich. the main ...,round trip hong kong munich . main reason fly ...,0
2,China,Air China,sydney beijing paris then rome beijing to sydn...,sydney beijing paris rome beijing sydney famil...,0
3,China,Air China,london to sydney return via beijing. a cheap f...,london sydney return beijing . cheap flight li...,0
4,China,Air China,beijing to shanghai. only one check in desk fo...,beijing shanghai . check desk standby passenge...,1


In [4]:
air['Review'][19]

'paris to beijing in business class on a 777 300 and return from shanghai to paris in business class on a340. seating on the 777 is outstanding with new seats large tv screens and a 2 2 2 configuration. on the a340 the configuration was the same but there were older seats smaller tv screens though i cannot complain about seating. on both legs staff were friendly and attentive. what is unsatisfying was the quality of the meals. i have seldom tasted such bland food and the presentation was horrible. before take off only orange juice and water were served. wines good selection of french wines were outstanding but must have been very expensive because there was no second serving nor were liquors offered at the end of the meal. another drawback was the total absence of reading materials in english or in french except a few china daily newspapers. a large proportion of the offered movies were oldies. if you are not interested in food if you bring your own reading materials air china is a goo

In [5]:
# Maximum length of the review
print(max([len(i.split()) for i in air['Review']]))
# Minimum length of the review
print(min([len(i.split()) for i in air['Review']]))

# Maximum length of the review_LS
print(max([len(i.split()) for i in air['Review_LS']]))
# Minimum length of the review_LS
print(min([len(i.split()) for i in air['Review_LS']]))

729
14
390
8


In [18]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

cuda


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt').to(device)
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

air['Sentiment_Bert'] = air['Review'].apply(lambda x: sentiment_score(x[:512]))


In [22]:
air.head()

,Country,Airline,Review,Review_LS,Sentiment,Sentiment_Bert
0,China,Air China,los angeles beijing return. food low quality s...,los angeles beijing return . food quality staf...,0,2
1,China,Air China,round trip from hong kong to munich. the main ...,round trip hong kong munich . main reason fly ...,0,1
2,China,Air China,sydney beijing paris then rome beijing to sydn...,sydney beijing paris rome beijing sydney famil...,0,2
3,China,Air China,london to sydney return via beijing. a cheap f...,london sydney return beijing . cheap flight li...,0,2
4,China,Air China,beijing to shanghai. only one check in desk fo...,beijing shanghai . check desk standby passenge...,1,1


In [24]:
tokenizer2 = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
model2 = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model2.to(device)

def sentiment_score(review):
    tokens = tokenizer2.encode(review, return_tensors='pt').to(device)
    result = model2(tokens)
    return int(torch.argmax(result.logits))+1

air['Sentiment_roberta'] = air['Review'].apply(lambda x: sentiment_score(x[:512]))


In [26]:
tokenizer3 = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model3 = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model3.to(device)

def sentiment_score(review):
    tokens = tokenizer3.encode(review, return_tensors='pt').to(device)
    result = model3(tokens)
    return int(torch.argmax(result.logits))+1

air['Sentiment_distilbert'] = air['Review'].apply(lambda x: sentiment_score(x[:512]))


In [28]:
air.head()

,Country,Airline,Review,Review_LS,Sentiment,Sentiment_Bert,Sentiment_roberta,Sentiment_distilbert
0,China,Air China,los angeles beijing return. food low quality s...,los angeles beijing return . food quality staf...,0,2,1,1
1,China,Air China,round trip from hong kong to munich. the main ...,round trip hong kong munich . main reason fly ...,0,1,1,1
2,China,Air China,sydney beijing paris then rome beijing to sydn...,sydney beijing paris rome beijing sydney famil...,0,2,1,1
3,China,Air China,london to sydney return via beijing. a cheap f...,london sydney return beijing . cheap flight li...,0,2,3,1
4,China,Air China,beijing to shanghai. only one check in desk fo...,beijing shanghai . check desk standby passenge...,1,1,1,1


air.to_csv('/content/drive/MyDrive/Airline Sentiment Analysis/output_df.csv',index=False)

In [1]:
# Importing EDA libraries

import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('output_df.csv')

In [3]:
data.head()

,Country,Airline,Review,Review_LS,Sentiment,Sentiment_Bert,Sentiment_roberta,Sentiment_distilbert
0,China,Air China,los angeles beijing return. food low quality s...,los angeles beijing return . food quality staf...,0,2,1,1
1,China,Air China,round trip from hong kong to munich. the main ...,round trip hong kong munich . main reason fly ...,0,1,1,1
2,China,Air China,sydney beijing paris then rome beijing to sydn...,sydney beijing paris rome beijing sydney famil...,0,2,1,1
3,China,Air China,london to sydney return via beijing. a cheap f...,london sydney return beijing . cheap flight li...,0,2,3,1
4,China,Air China,beijing to shanghai. only one check in desk fo...,beijing shanghai . check desk standby passenge...,1,1,1,1


In [5]:
# Checking unique values in the dataset

print(data['Sentiment'].unique())
print(data['Sentiment_Bert'].unique())
print(data['Sentiment_roberta'].unique())
print(data['Sentiment_distilbert'].unique())

[0 1]
[2 1 4 3 5]
[1 3 2]
[1 2]


In [6]:
# Changing the values of the other to match the sentiment values

data['Sentiment_Bert'] = data['Sentiment_Bert'].replace({1:0, 2:0, 3:1, 4:1, 5:1})
data['Sentiment_roberta'] = data['Sentiment_roberta'].replace({1:0, 2:1, 3:1})
data['Sentiment_distilbert'] = data['Sentiment_distilbert'].replace({1:0, 2:1})

In [7]:
data.head()

,Country,Airline,Review,Review_LS,Sentiment,Sentiment_Bert,Sentiment_roberta,Sentiment_distilbert
0,China,Air China,los angeles beijing return. food low quality s...,los angeles beijing return . food quality staf...,0,0,0,0
1,China,Air China,round trip from hong kong to munich. the main ...,round trip hong kong munich . main reason fly ...,0,0,0,0
2,China,Air China,sydney beijing paris then rome beijing to sydn...,sydney beijing paris rome beijing sydney famil...,0,0,0,0
3,China,Air China,london to sydney return via beijing. a cheap f...,london sydney return beijing . cheap flight li...,0,0,1,0
4,China,Air China,beijing to shanghai. only one check in desk fo...,beijing shanghai . check desk standby passenge...,1,0,0,0


In [11]:
# Checking Accuracy of the models using sklearn metrics

from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

print('Accuracy of the model using Sentiment Analysis is: ', accuracy_score(data['Sentiment'], data['Sentiment_Bert']))
print('Accuracy of the model using Sentiment Analysis is: ', accuracy_score(data['Sentiment'], data['Sentiment_roberta']))
print('Accuracy of the model using Sentiment Analysis is: ', accuracy_score(data['Sentiment'], data['Sentiment_distilbert']))
print('\n')

print('Confusion Matrix of the model using Sentiment Analysis is: \n ', confusion_matrix(data['Sentiment'], data['Sentiment_Bert']))
print('Confusion Matrix of the model using Sentiment Analysis is: \n', confusion_matrix(data['Sentiment'], data['Sentiment_roberta']))
print('Confusion Matrix of the model using Sentiment Analysis is: \n', confusion_matrix(data['Sentiment'], data['Sentiment_distilbert']))
print('\n')

print('Classification Report of the model using Sentiment Analysis is: \n', classification_report(data['Sentiment'], data['Sentiment_Bert']))
print('Classification Report of the model using Sentiment Analysis is: \n', classification_report(data['Sentiment'], data['Sentiment_roberta']))
print('Classification Report of the model using Sentiment Analysis is: \n', classification_report(data['Sentiment'], data['Sentiment_distilbert']))

Accuracy of the model using Sentiment Analysis is:  0.8575562278048139
Accuracy of the model using Sentiment Analysis is:  0.8288833355254505
Accuracy of the model using Sentiment Analysis is:  0.8638037616730238


Confusion Matrix of the model using Sentiment Analysis is: 
  [[6194 1513]
 [ 653 6846]]
Confusion Matrix of the model using Sentiment Analysis is: 
 [[5507 2200]
 [ 402 7097]]
Confusion Matrix of the model using Sentiment Analysis is: 
 [[6992  715]
 [1356 6143]]


Classification Report of the model using Sentiment Analysis is: 
               precision    recall  f1-score   support

           0       0.90      0.80      0.85      7707
           1       0.82      0.91      0.86      7499

    accuracy                           0.86     15206
   macro avg       0.86      0.86      0.86     15206
weighted avg       0.86      0.86      0.86     15206

Classification Report of the model using Sentiment Analysis is: 
               precision    recall  f1-score   support

    